In [ ]:
"""
Real-time finger writing on webcam screen using Python, OpenCV and MediaPipe.

Controls:
 - Draw by holding up only the INDEX finger (other fingers down).
 - Move cursor (no drawing) when INDEX + MIDDLE fingers are up.
 - Press 'c' to clear canvas.
 - Press 's' to save the current canvas as 'drawing_<timestamp>.png'.
 - Press 'q' to quit.

Dependencies:
 pip install opencv-python mediapipe numpy
"""

import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque

# --- Config
MAX_POINTS = 1024        # maximum points to keep in a single stroke
SMOOTHING = 0.6          # smoothing factor [0..1], higher => smoother
BRUSH_THICKNESS = 6
BRUSH_COLOR = (0, 0, 255)  # BGR (red)
ERASE_KEY = ord('c')
SAVE_KEY = ord('s')
QUIT_KEY = ord('q')

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

def fingers_up(hand_landmarks):
    """
    Return a tuple of booleans (thumb, index, middle, ring, pinky) indicating whether each finger is up.
    Works for a single hand (assumes right-hand-ish orientation but is okay for our simple rule).
    """
    lm = hand_landmarks.landmark
    tips_ids = [4, 8, 12, 16, 20]
    fingers = []

    # Thumb: compare tip with IP joint in x-axis (handedness can flip; this is a heuristic)
    # Use x comparison relative to wrist for direction
    wrist_x = lm[0].x
    thumb_tip_x = lm[4].x
    thumb_ip_x = lm[3].x
    # simple rule:
    fingers.append(thumb_tip_x < thumb_ip_x if wrist_x < 0.5 else thumb_tip_x > thumb_ip_x)

    # Other fingers: tip y < pip y -> finger is up (y increases downward)
    for id in [8, 12, 16, 20]:
        fingers.append(lm[id].y < lm[id - 2].y)
    return tuple(fingers)


def scaled_point(lm, frame_w, frame_h):
    """Convert normalized landmark (x,y) to pixel coords (int)."""
    return int(lm.x * frame_w), int(lm.y * frame_h)


def main():
    cap = cv2.VideoCapture(0)
    time.sleep(0.5)
    if not cap.isOpened():
        print("Unable to open webcam. If you're running in a headless environment, webcam won't be available.")
        return

    ret, frame = cap.read()
    if not ret:
        print("Can't read from webcam.")
        return
    h, w = frame.shape[:2]

    # Canvas to draw on (same size as frame)
    canvas = np.zeros_like(frame)

    # Store last position and a queue of points for a stroke
    last_x, last_y = None, None
    stroke_pts = deque(maxlen=MAX_POINTS)

    prev_point = None
    smoothing_point = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # mirror image for natural drawing
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)

        draw_mode = False  # whether we should draw this frame
        cursor = None

        if result.multi_hand_landmarks:
            hand_landmarks = result.multi_hand_landmarks[0]
            lm = hand_landmarks.landmark

            # get finger states
            thumb, index, middle, ring, pinky = fingers_up(hand_landmarks)

            # index tip coords
            ix, iy = scaled_point(lm[8], w, h)
            cursor = (ix, iy)

            # rule: draw if index is up and middle is down
            if index and not middle:
                draw_mode = True
            else:
                draw_mode = False

            # draw landmark overlay small (for user feedback)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Drawing logic
        if cursor is not None:
            # smoothing: exponential moving average between previous smoothing_point and current cursor
            if smoothing_point is None:
                smoothing_point = cursor
            else:
                sx = int(smoothing_point[0] * (1 - SMOOTHING) + cursor[0] * SMOOTHING)
                sy = int(smoothing_point[1] * (1 - SMOOTHING) + cursor[1] * SMOOTHING)
                smoothing_point = (sx, sy)

            px, py = smoothing_point

            if draw_mode:
                # add point to stroke
                if prev_point is None:
                    prev_point = (px, py)
                    stroke_pts.append(prev_point)
                else:
                    # only add if moved sufficiently to reduce redundant points
                    dx = px - prev_point[0]
                    dy = py - prev_point[1]
                    if (dx * dx + dy * dy) > 2:
                        stroke_pts.append((px, py))
                        prev_point = (px, py)
            else:
                # if we were drawing and now stopped -> finalize stroke onto canvas
                if len(stroke_pts) > 1:
                    pts = np.array(stroke_pts, dtype=np.int32)
                    cv2.polylines(canvas, [pts], False, BRUSH_COLOR, thickness=BRUSH_THICKNESS, lineType=cv2.LINE_AA)
                    stroke_pts.clear()
                prev_point = None

            # Draw a small cursor circle on the live frame
            cv2.circle(frame, (px, py), 8, (0, 255, 0) if draw_mode else (255, 0, 0), -1)

        # Temporary live stroke preview (draw current stroke on top of canvas but not yet committed)
        preview = canvas.copy()
        if len(stroke_pts) > 1:
            pts = np.array(stroke_pts, dtype=np.int32)
            cv2.polylines(preview, [pts], False, BRUSH_COLOR, thickness=BRUSH_THICKNESS, lineType=cv2.LINE_AA)

        # Combine preview with webcam frame: show webcam in background and drawing overlay on top with alpha
        alpha = 0.6
        overlay = cv2.addWeighted(frame, 1.0, np.zeros_like(frame), 0, 0)  # frame copy
        # put the drawing as semi-transparent on top of the webcam frame:
        combined = cv2.addWeighted(overlay, 1.0, preview, alpha, 0)

        # helpful instructions text
        cv2.putText(combined, "Draw: index up & middle down | c: clear | s: save | q: quit",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (230, 230, 230), 2, cv2.LINE_AA)

        cv2.imshow("Finger Paint (Press q to quit)", combined)

        key = cv2.waitKey(1) & 0xFF
        if key == QUIT_KEY:
            break
        elif key == ERASE_KEY:
            canvas[:] = 0
            stroke_pts.clear()
            prev_point = None
            smoothing_point = None
            print("Canvas cleared.")
        elif key == SAVE_KEY:
            fname = f"drawing_{int(time.time())}.png"
            # merge canvas onto a white bg for a clean save
            white_bg = 255 * np.ones_like(canvas)
            saved = cv2.addWeighted(white_bg, 1.0, canvas, 1.0, 0)
            cv2.imwrite(fname, saved)
            print(f"Saved {fname}")

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)
canvas = None
prev_x, prev_y = 0, 0
drawing = False  # Start/stop drawing toggle

print("Press 'c' to clear, 's' to save, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    if canvas is None:
        canvas = np.zeros_like(frame)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        lm = hand_landmarks.landmark

        # Get index fingertip (landmark 8)
        x = int(lm[8].x * w)
        y = int(lm[8].y * h)

        if prev_x == 0 and prev_y == 0:
            prev_x, prev_y = x, y

        # Draw line between previous and current points
        cv2.line(canvas, (prev_x, prev_y), (x, y), (0, 0, 255), 5)

        prev_x, prev_y = x, y

        # Draw a circle at the fingertip
        cv2.circle(frame, (x, y), 8, (0, 255, 0), -1)

    else:
        prev_x, prev_y = 0, 0  # Reset when hand not detected

    # Overlay the canvas on the frame
    frame = cv2.addWeighted(frame, 0.7, canvas, 0.8, 0)

    cv2.putText(frame, "Index Finger Drawing | c: clear | s: save | q: quit",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("Finger Draw (Index Finger Only)", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        canvas = np.zeros_like(frame)
    elif key == ord('s'):
        filename = f"drawing_{int(time.time())}.png"
        cv2.imwrite(filename, canvas)
        print(f"Saved {filename}")

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

SMOOTHING_FACTOR = 0.5
MIN_DISTANCE = 3
INTERPOLATION_STEPS = 5
MAX_POINTS = 64

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

canvas = None
prev_x, prev_y = 0, 0
smoothed_x, smoothed_y = 0, 0
stroke_buffer = deque(maxlen=MAX_POINTS)

def fingers_up(hand_landmarks):
    lm = hand_landmarks.landmark
    wrist_x = lm[0].x
    thumb_up = (lm[4].x < lm[3].x) if (wrist_x < 0.5) else (lm[4].x > lm[3].x)
    index_up = lm[8].y < lm[6].y
    middle_up = lm[12].y < lm[10].y
    ring_up = lm[16].y < lm[14].y
    pinky_up = lm[20].y < lm[18].y
    return thumb_up, index_up, middle_up, ring_up, pinky_up

def interpolate_points(p1, p2, steps):
    x1, y1 = p1
    x2, y2 = p2
    x_step = (x2 - x1) / (steps + 1)
    y_step = (y2 - y1) / (steps + 1)
    points = []
    for i in range(1, steps + 1):
        x = int(x1 + x_step * i)
        y = int(y1 + y_step * i)
        points.append((x, y))
    return points

def normalize_coordinates(x, y, frame_width, frame_height):
    padding = 50
    x = max(padding, min(frame_width - padding, x))
    y = max(padding, min(frame_height - padding, y))
    return x, y

print("🖊️ Air Pen Mode Active")
print("- Raise index finger to draw")
print("- Raise all 5 fingers to erase")
print("Press 'c' to clear | 's' to save | 'q' to quit")
print(f"Canvas size: {actual_width}x{actual_height}")

cv2.namedWindow("Air Pen", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Air Pen", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    if canvas is None:
        canvas = np.zeros((h, w, 3), dtype=np.uint8)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    mode = "none"
    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        thumb_up, index_up, middle_up, ring_up, pinky_up = fingers_up(hand_landmarks)

        ix = int(hand_landmarks.landmark[8].x * w)
        iy = int(hand_landmarks.landmark[8].y * h)
        ix, iy = normalize_coordinates(ix, iy, w, h)

        if smoothed_x == 0 and smoothed_y == 0:
            smoothed_x, smoothed_y = ix, iy
        else:
            smoothed_x = int(smoothed_x * (1 - SMOOTHING_FACTOR) + ix * SMOOTHING_FACTOR)
            smoothed_y = int(smoothed_y * (1 - SMOOTHING_FACTOR) + iy * SMOOTHING_FACTOR)

        if thumb_up and index_up and middle_up and ring_up and pinky_up:
            mode = "erase"
        elif index_up and not (middle_up or ring_up or pinky_up or thumb_up):
            mode = "draw"

        if mode != "none":
            current_point = (smoothed_x, smoothed_y)
            
            if prev_x == 0 and prev_y == 0:
                prev_x, prev_y = smoothed_x, smoothed_y
                if mode == "draw":
                    stroke_buffer.append(current_point)
            else:
                dist = np.sqrt((smoothed_x - prev_x)**2 + (smoothed_y - prev_y)**2)
                
                if dist >= MIN_DISTANCE:
                    if mode == "draw":
                        if dist > MIN_DISTANCE * 4:
                            interp_points = interpolate_points((prev_x, prev_y), current_point, INTERPOLATION_STEPS)
                            for p in interp_points:
                                stroke_buffer.append(p)
                                if len(stroke_buffer) >= 2:
                                    p1 = stroke_buffer[-2]
                                    p2 = stroke_buffer[-1]
                                    cv2.line(canvas, p1, p2, (0, 0, 255), 2, cv2.LINE_AA)
                        else:
                            stroke_buffer.append(current_point)
                            if len(stroke_buffer) >= 2:
                                cv2.line(canvas, 
                                        stroke_buffer[-2], 
                                        stroke_buffer[-1], 
                                        (0, 0, 255), 2, cv2.LINE_AA)
                    
                    elif mode == "erase":
                        cv2.circle(canvas, (smoothed_x, smoothed_y), 20, (0, 0, 0), -1, cv2.LINE_AA)
                        if dist > MIN_DISTANCE * 4:
                            interp_points = interpolate_points((prev_x, prev_y), current_point, INTERPOLATION_STEPS)
                            for px, py in interp_points:
                                cv2.circle(canvas, (px, py), 20, (0, 0, 0), -1, cv2.LINE_AA)
                    
                    prev_x, prev_y = smoothed_x, smoothed_y
        else:
            prev_x, prev_y = 0, 0
            stroke_buffer.clear()

        color = (0, 255, 0) if mode == "draw" else ((255, 255, 255) if mode == "erase" else (255, 0, 0))
        size = 8 if mode == "draw" else (20 if mode == "erase" else 8)
        cv2.circle(frame, (smoothed_x, smoothed_y), size, color, -1, cv2.LINE_AA)

    else:
        prev_x, prev_y = 0, 0
        stroke_buffer.clear()
        smoothed_x, smoothed_y = 0, 0

    frame = cv2.addWeighted(frame, 0.7, canvas, 0.8, 0)

    cv2.putText(frame, "Air Pen | Draw: Index Up | Erase: All Fingers Up | c: clear | s: save | q: quit",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Air Pen", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        canvas = np.zeros_like(frame)
        stroke_buffer.clear()
    elif key == ord('s'):
        filename = f"drawing_{int(time.time())}.png"
        cv2.imwrite(filename, canvas)
        print(f"Saved {filename}")

cap.release()
cv2.destroyAllWindows()